<a href="https://colab.research.google.com/github/KaushalJogi2002/Flowise/blob/main/Phi2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q langchain
!pip install -q torch
!pip install -q sentence-transformers
!pip install -q datasets
!pip install -q faiss-cpu
!pip uninstall -y transformers
!pip install git+https://github.com/huggingface/transformers
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.4/246.4 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.5/149.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 39.3 MB/s eta 0:00:00
Found existing installation: transformers 4.37.2
Uninstalling transformers-4.37.2:
  Successfully uninst

In [2]:
!pip install -q pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 6.0 MB/s eta 0:00:00


In [3]:
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForQuestionAnswering , AutoModelForCausalLM
from transformers import AutoTokenizer, pipeline
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import accelerate

In [4]:
loader = PyPDFLoader("/content/Code.pdf")
data = loader.load_and_split()

In [5]:
# Create an instance of the RecursiveCharacterTextSplitter class with specific parameters.
# It splits text into chunks of 1000 characters each with a 150-character overlap.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)

# 'data' holds the text you want to split, split the text into documents using the text splitter.
docs = text_splitter.split_documents(data)

In [6]:
docs[10]

Document(page_content='T echnology ,\nSolapur ,\nbased\non\nthe\npr o vided\ninformation:\nKnowledge\nBase\nfor\nWI TBO T:\n1.\nPlacement\nInformation:\n●\n1.1\nOv er view:\n●\nThe\nplacement\ntr ack er\nfor\nthe\nacademic\ny ear\n2022-2023\nshowcases\ncompanies,\njob\noff ers,\nsalar y\npackages,\nand\ndisciplines\nfor\nplacements\nat\nthe\ninstitute.\n●\n1.2\nNotable\nCompanies:\n●\nAqua\nChill\nSystems\nIndia\nP vt.\nLtd.,\nAssam\nRiﬂes,\nA tlas\nCopco\n(India)\nLtd.,\nA tlas\nCopco\nGECI A,\nA ut omatic\nInfotech\nPriv ate\nLimited,\nWhitekla y ,\nand\nmor e.\n●\n1.3\nDisciplines\nand\nP ackages:\n●\nDisciplines\ninclude\nMechanical\nEngineering\n(MECH),\nCivil\nEngineering\n(CIVIL),\nComputer\nScience/Information\nT echnology\n(CSE/I T),\nElectr onics\nand\nT elecommunication\n(EN T C ),\nElectrical\nEngineering\n(ELN),\netc.\n●\nSalar y\npackages\nr ange\nfr om\n1.44\nLP A\nt o\n12.5\nLP A.', metadata={'source': '/content/Code.pdf', 'page': 5})

In [7]:
# Define the path to the pre-trained model you want to use
modelPath = "sentence-transformers/all-MiniLM-l6-v2"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cpu'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
text = "This is a test document."
query_result = embeddings.embed_query(text)
query_result[:3]

[-0.03833850473165512, 0.1234646886587143, -0.028642967343330383]

In [9]:
db = FAISS.from_documents(docs, embeddings)

In [10]:
question = "Where should i lodge my complaint related to traffic police?"
searchDocs = db.similarity_search(question)
print(searchDocs[0].page_content)

(Regulation
of
Admission
and
Fees)
Act,
2015.
3.
Fee
Payment
Process:
●
3.1
Online
Fee
Payment:
●
Utilize
the
SBI
Collect
portal
for
online
fee
payment.
●
Payment
options
include
Net
Banking,
Card
Payments,
and
other
modes.
●
Fetch
payment
details
using
the
provided
Enrollment
Number.
●
3.2
Confirmation
and
Receipt:
●
Take
a
printout
of
the
e-receipt
after
successful
payment.
●
Contact
the
office
for
confirmation
of
admission/provisional
admission.
●
3.3
Transaction
Limits
and
Troubleshooting:
●
Be
aware
of
transaction
limits
and
troubleshoot
issues
following
instructions
under
"Customer
Support."
4.
Documentation:
●
4.1
Admission
Form
Submission:
●
After
payment,
collect
the
admission
form
from
the
Library.
●
Submit
the
duly
filled
form
in
the
office.
●
4.2
Google
Form
Submission:
●
Fill
the
Google
Form
with
fee
payment
details
provided
at
the
specified
link.


In [11]:
!pip install accelerate

In [12]:
import accelerate
accelerate.__version__

'0.27.2'

In [13]:

model_name = "mistralai/Mistral-7B-Instruct-v0.2"


# Load the tokenizer associated with the specified model
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", max_length=1024,
                                          trust_remote_code=True)




tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [14]:
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", torch_dtype="auto", device_map="cuda", trust_remote_code=True)


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [15]:
# Define a question-answering pipeline using the model and tokenizer
'''question_answerer = pipeline(
    "question-answering",
    model=model_name,
    tokenizer=tokenizer,
    return_tensors='pt',
    trust_remote_code=True
)'''

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=500)
# Create an instance of the HuggingFacePipeline, which wraps the question-answering pipeline
# with additional model-specific arguments (temperature and max_length)
llm = HuggingFacePipeline(
    pipeline=pipe,
    model_kwargs={"temperature": 0.7, "max_length": 512},
)

In [16]:
# Create a retriever object from the 'db' with a search configuration where it retrieves up to 4 relevant splits/documents.
retriever = db.as_retriever(search_kwargs={"k": 3})

# Create a question-answering instance (qa) using the RetrievalQA class.
# It's configured with a language model (llm), a chain type "refine," the retriever we created, and an option to not return source documents.
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="refine", retriever=retriever, return_source_documents=False)

In [17]:
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

'en_US.UTF-8'

In [19]:
question = "What is fees structureof wit?"
result = qa.invoke({"query": question})
print(result["result"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Based on the context information provided, the fees structure for Undergraduate Programs at Walchand Institute of Technology (WIT) for the academic year 2023-24 is as follows:
- Open Category: Rs. 1,10,000/-
- EBC/OBC/EWS: Rs. 63,000/-
- SC & ST: Rs. 1,533/-
- VJNT/SBC: Rs. 15,620/-
- PMSSS: Rs. 1,533/-
The academic session for the final year of B.Tech program 2023-2024 starts on 17th August 2023. Students are required to complete provisional admission by paying the annual fees in a single installment.
The fee structure for Postgraduate programs is as follows:
- SC Scholarship: Rs. 1,533/-
It is advisable for students to check for any updates or revisions to the fee structure, especially for the provisional fees in the academic year 2023-24. The fee structure is crucial for students to understand the financial obligations associated with their education at Walchand Institute of Technology.
Important Note: Any upward revision of fees is binding and mandatory as per the Maharashtra Unai